In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow
import os
import io
from PIL import Image

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
#import tflite_runtime.interpreter as tflite
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model

In [2]:
img_height = 256
img_width = 256
class_folder_paths = ['C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/'+x for x in os.listdir('C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/')]
class_folder_paths

['C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Afghan',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/African Wild Dog',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Airedale',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/American Hairless',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/American Spaniel',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Basenji',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Basset',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Beagle',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bearded Collie',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bermaise',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bichon Frise',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Blenheim',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bloodhound',
 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/B

In [3]:
for class_folder_path in class_folder_paths:
    print('{0}:'.format(class_folder_path), ' ', len(os.listdir(class_folder_path)))


C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Afghan:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/African Wild Dog:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Airedale:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/American Hairless:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/American Spaniel:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Basenji:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Basset:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Beagle:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bearded Collie:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bermaise:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bichon Frise:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Blenheim:   10
C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/Bloodhound:   10
C:/Users/krzys/Desktop/Polibuda/se

In [4]:
data_dir = 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/'
val_dir = 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/'


train_dir=os.path.join(data_dir, 'train')
# train_dir=os.path.join(data_dir, 'train2')
val_dir=os.path.join(val_dir, 'test')

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  image_size=(img_height, img_width),
  batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dir,
  image_size=(img_height, img_width),
  batch_size=32)

Found 15856 files belonging to 4 classes.
Found 700 files belonging to 70 classes.


In [6]:
TRAIN_DIR = 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/train/'
# TRAIN_DIR = 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/train2/'
VAL_DIR = 'C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/archive/test/'

train_data_gen = ImageDataGenerator(horizontal_flip = True,
                                    rotation_range=20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.2)

train_generator = train_data_gen.flow_from_directory(TRAIN_DIR,
                                                     target_size = (224, 224),
                                                     color_mode = 'rgb',
                                                     batch_size = 32,
                                                     class_mode ='categorical',
                                                     shuffle = True)

val_data_gen = ImageDataGenerator()

val_generator = val_data_gen.flow_from_directory(VAL_DIR,
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = False)


labels = train_generator.class_indices
class_mapping = dict((v,k) for k,v in labels.items())
class_mapping
*labels2, = labels.values()



Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [16]:
#Tworzenie zbiorów obrazów do treningu i walidacji
train_ds = tf.keras.preprocessing.image_dataset_from_directory( train_dir, labels='inferred', label_mode='int', image_size=(img_height, img_width), batch_size=32)
val_ds = tf.keras.preprocessing.image_dataset_from_directory( val_dir, labels='inferred', label_mode='int', image_size=(img_height, img_width), batch_size=32)

Found 7946 files belonging to 70 classes.
Found 700 files belonging to 70 classes.


In [8]:
#Tworzenie zbiorów obrazów do treningu i walidacji
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir)

val_ds = tf.keras.utils.image_dataset_from_directory(
  val_dir)

Found 15856 files belonging to 4 classes.
Found 700 files belonging to 70 classes.


In [9]:
#Pokazanie części obrazów, aby sprawdzić czy się załadowały
plt.figure(figsize=(100, 100))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_mapping)
    plt.axis("off")

In [10]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break



AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


normalization_layer = layers.Rescaling(1./255)


(32, 256, 256, 3)
(32,)


In [11]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

InvalidArgumentError: Input is empty.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext]

In [12]:
num_classes = len(class_mapping)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        1

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/10


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
img_path = "D:/PENDRIVE/CV/dog3.jpg"
img = Image.open(img_path)
plt.imshow(img)
plt.show()


img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)


#img = tf.keras.utils.load_img(

#    sunflower_path, target_size=(img_height, img_width))

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [ ]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

In [ ]:
print(format(class_mapping[np.arg(score[0])], 100 * score[0]))
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_mapping[np.argmax(score)], 100 * np.max(score))
)
#print(decode_predictions(predictions, top=3)[0])

In [ ]:
#model = tf.keras.applications.resnet50.ResNet50()
#model = first_model
predictions = model.predict(img_preprocessed)

In [ ]:
print(decode_predictions(predictions, top=3)[0])

In [ ]:
print(predictions)

In [ ]:
first_model = model

In [ ]:
base_model = model
# add a global spatial average pooling layer
x = base_model.output
#x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 7 classes
predictions = Dense(70, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
modelmodel = tf.keras.models.load_model('C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/model')
modelmodel.summary()

In [ ]:
tf.math.confusion_matrix(
    labels,
    predictions,
    num_classes=None,
    weights=None,
    dtype=tf.dtypes.int32,
    name=None
)

In [ ]:
y=[]
for i in 70:
    y.append(1)

In [ ]:

confusion = tf.math.confusion_matrix(labels=labels2, predictions=predictions)
print(confusion)